In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import numpy.linalg as la
import scipy as sp
from scipy.linalg import block_diag
import time
import itertools
from math import pi, sqrt

In [3]:
def initialize_wf(N,N_pt):
    coords=np.sort(random.sample(range(N),N_pt))
    n_occ=np.zeros((N,))        
    n_occ[coords]=np.ones((N_pt,))        
    n_pos = np.zeros((N,),dtype=int)        
    n_pos[coords] = np.arange(1,N_pt+1)
    
    return n_occ, n_pos, coords

def initialize_wf_region(n, n_e, inds_A1, n_A1,inds_A2, n_A2):
    
    inds_A = np.append(inds_A1,inds_A2).tolist()
    inds_A1 = inds_A1.tolist()
    inds_A2 = inds_A2.tolist()
    inds_outsideA = np.arange(n)
    if inds_A1 == []:
        inds_outsideA = np.delete(inds_outsideA, (inds_A2)).tolist()
    elif inds_A2 == []:
        inds_outsideA = np.delete(inds_outsideA, (inds_A1)).tolist()
    else:
        inds_outsideA = np.delete(inds_outsideA, inds_A).tolist()
    n_remain = n_e-n_A1-n_A2

    assert n_A1 <= len(inds_A1), 'n_A1 > subsystem A1 size'
    assert n_A2 <= len(inds_A2), 'n_A2 > subsystem A2 size'
    assert n_remain <= n-len(inds_A1)-len(inds_A2), 'n_B > subsystem B size'

    config = [0]*n_e       
    config[:n_A1] = np.sort(random.sample(inds_A1, n_A1))
    config[n_A1:n_A1+n_A2] = np.sort(random.sample(inds_A2, n_A2))
    config[n_A1+n_A2:] = np.sort(random.sample(inds_outsideA, n_remain))
    np.random.shuffle(config) 
    
    n_loc = np.zeros((n,))        
    n_loc[config] = np.ones((n_e,))    
    n_ind = np.zeros((n,),dtype=int)        
    n_ind[config] = np.arange(1,n_e+1)
    
    return n_loc, n_ind, config[:n_e]

def wf_gen(N,N_pt,BC,t):
    hop = np.diag(np.ones(N-1) + 0j,1)
    hop[N-1,0] = BC
    H_t = -t*(np.matrix(hop)+np.matrix(hop).H)/2  ###.H gives transpose
    energies, evecs= np.linalg.eigh(H_t)
    exact_gs_energy=  np.sum(energies[:N_pt])
    #print("exact gs energy is", exact_gs_energy)
    
    return evecs[:,:N_pt]

def wf_SSH_gen(N,N_pt,BC,t1,t2):
    h1=np.ones(N-1)
    h1[0::2]=0
    h2=np.ones(N-1)
    h2[1::2]=0
    hop= np.diag(t1*h1+t2*h2+0j,1)
    hop[N-1,0]= t1*BC
    H_t= -(hop+ np.matrix(hop).H)/2 
    energies, evecs= np.linalg.eigh(H_t)
    return evecs[:,:N_pt]

def WF_MAT(wf_gen, config):
    n_e = len(config)
    wf = np.zeros((n_e,n_e),dtype = "complex")
    wf = np.transpose(wf_gen[config,:])
     
    return wf

def exact_renyi_calc(r,GA,epsilon=1e-9):
    chi0, _ =np.linalg.eigh(GA)
    chi1=chi0[np.nonzero(np.abs(chi0)>epsilon)]
    chi2=chi1[np.nonzero(np.abs(chi1-1)>epsilon)] #/np.sum(chi1[np.nonzero(np.abs(chi1-1)>epsilon)])
    return np.sum(np.log((1-chi2)**r+chi2**r))/(1-r) #np.sum(np.log((1-chi2)**r+chi2**r))/(1-r) #Twice as large....
    #return np.log(np.sum(chi2**r))/(1-r) 

In [4]:
def exact_Renyi_neg_calc(r,N,Na,Nb,V1,epsilon=1e-9):
    '''computes moments of partial transpose tr(\rho^T \rho^T\dag ... )
    for 1d free fermions ''' 
    k_sw=np.arange((-(r-1)/2),(r-1)/2+0.1,1)
    Zk=0
    if (r % 2) == 0:
        delta=pi
    else:
        delta= pi*(r-1)/r
#     delta=pi

    Nflip=Na+Nb
    N1=int((N-Nflip)/2)

    for i_k in range(len(k_sw)):
#         op_k=block_diag(np.eye(N1),np.exp(1j*2*pi/r*k_sw[i_k])*np.eye(Na),np.exp(1j*delta-1j*2*pi/r*k_sw[i_k])*np.eye(Nb),np.eye(N1))
        op_k=block_diag(np.exp(1j*2*pi/r*k_sw[i_k])*np.eye(Na),np.exp(1j*delta-1j*2*pi/r*k_sw[i_k])*np.eye(Nb),np.eye(N-Nflip))    
        Zk +=np.real(np.log( np.linalg.det( np.dot(np.dot(np.matrix(V1).H,op_k),V1) )))
    return Zk

In [5]:
def VMC_equal_number(numconfig,check_step,N,N_pt,wf_gen,wf_r,n_occ_r,coords_r,inds_A):


    move_attempted = 0
    move_accepted = 0
    
    r = wf_r.shape[2]
    
    wf_inv_r = np.zeros(wf_r.shape,dtype=np.complex128)
    for i_r in range(r):
        wf_inv_r[:,:,i_r]=np.linalg.inv(wf_r[:,:,i_r])
        
    count=0 # counter for energy
    min_state=5
    ratio_0=1.0+0.0j
    
    Na = np.zeros((1,r))
    
    fraction=np.zeros(np.max(inds_A)+2) #a count for each potential particle number in the region 
    
    # possible steps
    step_abs=np.arange(1,Ns+1) #Ns?
    step_vals=np.sort(np.concatenate((-step_abs,step_abs),axis=0)).tolist()

    for state in range(numconfig+min_state):
        for step in range(0,check_step):               #Samples observables after check_step iterations
        
            for i_r in range(r):
                for moved_elec in range(N_pt):
                    move_attempted=move_attempted+1

                    # random walk of Ns steps left or right
                    stepx=random.sample(step_vals,1)[0]
                    ptcls_x= np.mod( coords_r[moved_elec,i_r]+stepx, N) # new configuration

                    if n_occ_r[ptcls_x,i_r]==1:
                        continue

                    u_0 = np.transpose(wf_gen[ptcls_x,:] - wf_gen[coords_r[moved_elec,i_r],:])
                    #pt_wf_1=np.transpose(wf_gen[ptcls_x,:])

                    rel =  np.dot(wf_gen[ptcls_x,:],wf_inv_r[moved_elec,:,i_r].reshape((N_pt,1)))
                    #rel=np.dot(inv_wf[moved_elec,:],pt_wf_1)

                    alpha=min(1, np.abs(rel)**2)

                    random_num=random.random()

                    if random_num <= alpha:
                        move_accepted += 1

                        #Sherman-Morisson Formula for updating inverse matrix
                        v=np.zeros((N_pt,1))
                        v[moved_elec]=1
                        num = np.dot(np.dot(np.dot(wf_inv_r[:,:,i_r],u_0),v.T),wf_inv_r[:,:,i_r])
                        denom = (1+np.dot(v.T,np.dot(wf_inv_r[:,:,i_r],u_0)))
                        wf_inv_r[:,:,i_r] = wf_inv_r[:,:,i_r] - num/denom 
                        wf_r[:,moved_elec,i_r] = np.reshape(np.transpose(wf_gen[ptcls_x,:]),(N_pt,))

                        delta = np.zeros(N)
                        delta[coords_r[moved_elec,i_r]] = -1
                        delta[ptcls_x]=1

                        n_occ_r[:,i_r] = n_occ_r[:,i_r] + delta
                        n_pos_r[:,i_r] = n_pos_r[:,i_r] + (moved_elec + 1)*delta
                        coords_r[moved_elec,i_r] = ptcls_x 
                        
                x=np.argwhere(n_occ_r[:,i_r]>0)
                assert len(x)== N_pt, 'no of ptcle is %d' % (len(x))

        if state> (min_state-1):
            pt_num_inside= np.sum(n_occ_r[inds_A,:],axis=0)
            
            if np.max(pt_num_inside)==np.min(pt_num_inside) :
                fraction[np.max(pt_num_inside)] += 1
                
                
        if (state%500) ==0:
            for i_r in range(r):
                wf_inv_r[:,:,i_r]=np.linalg.inv(wf_r[:,:,i_r])

    acc_ratio=move_accepted/move_attempted
    fraction = fraction/(numconfig)        #Normalization needed?
    
    #print("fraction acceptance rate=", acc_ratio)
    return fraction

In [6]:
def VMC_amplitude_ratio(numconfig,check_step,N,N_pt,wf_gen,wf_r,n_occ_r,n_pos_r,coords_r,inds_A,N_pt_A):

    move_attempted = 0
    move_accepted = 0
    
    r = wf_r.shape[2]
    
    wf_inv_r = np.zeros(wf_r.shape,dtype=np.complex128)
    #number_pt_inside_A= np.sum(n_occ_r[inds_A,0],axis=0)
    wf_inds= np.zeros((N_pt_A,r),dtype=int)
    for i_r in range(r):
        wf_inv_r[:,:,i_r]=np.linalg.inv(wf_r[:,:,i_r])
        pt_num_inside = np.argwhere( n_occ_r[inds_A,i_r]>0 )
        pt_num_inside = np.reshape( pt_num_inside, (N_pt_A,)).tolist()
        wf_inds[:,i_r]=( n_pos_r[ inds_A[pt_num_inside], i_r] )-1   #Used to be sorted
    
    wf_swap = np.zeros(wf_r.shape,dtype=np.complex128)
    up_one = (np.arange(r)+1)%r
    for i_r in range(r):
    # r permutation of subsystem indices
        wf_swap[:,:,i_r] = np.copy(wf_r[:,:,i_r])
        wf_swap[:,wf_inds[:,i_r],i_r] = np.copy(wf_r[:,wf_inds[:,up_one[i_r]],up_one[i_r]]) #Shifting configurations with the next wf in sequence

    inside_A = 0
    step_abs=np.arange(1,Ns+1) 
    step_vals=np.sort(np.concatenate((-step_abs,step_abs),axis=0)).tolist()
        
    count=0 # counter for entropy
    min_state=5
    
    ent_ratio=np.zeros(int(numconfig),dtype=np.float64) 
    
    for state in range(numconfig+min_state):
        for step in range(0,check_step):               #Samples observables after check_step iterations
        
            for i_r in range(r):
                for moved_elec in range(N_pt):
                    move_attempted=move_attempted+1
                    
                    stepx=random.sample(step_vals,1)[0]
                    ptcls_x= np.mod( coords_r[moved_elec,i_r]+stepx, N) # new configuration

                    if n_occ_r[ptcls_x,i_r]==1:
                        continue

                    val_orig = np.min( np.abs(inds_A- coords_r[moved_elec,i_r]) )
                    val_dest = np.min( np.abs(inds_A-ptcls_x) )
                    if val_orig==0 and val_dest==0 :
                        inside_A=1
                        val_1 = np.min(np.abs(wf_inds[:,i_r]-moved_elec))
                        ind_1 = np.argmin(np.abs(wf_inds[:,i_r]-moved_elec))
                        assert val_1==0 , 'wrong inds_A for wf'
                    elif val_orig!=0 and val_dest!=0 :
                        inside_A=0
                    else:
                        continue

                    u_0 = np.transpose(wf_gen[ptcls_x,:] - wf_gen[coords_r[moved_elec,i_r],:])
                    #pt_wf_1=np.transpose(wf_gen[ptcls_x,:])

                    rel =  np.dot(wf_gen[ptcls_x,:],wf_inv_r[moved_elec,:,i_r].reshape((N_pt,1)))
                    #rel=np.dot(inv_wf[moved_elec,:],pt_wf_1)

                    alpha=min(1, np.abs(rel)**2)

                    random_num=random.random()

                    if random_num <= alpha:
                        move_accepted += 1

                        #Sherman-Morisson Formula for updating inverse matrix
                        v=np.zeros((N_pt,1))
                        v[moved_elec]=1
                        num = np.dot(np.dot(np.dot(wf_inv_r[:,:,i_r],u_0),v.T),wf_inv_r[:,:,i_r])
                        denom = (1+np.dot(v.T,np.dot(wf_inv_r[:,:,i_r],u_0)))
                        wf_inv_r[:,:,i_r] = wf_inv_r[:,:,i_r] - num/denom 
                        wf_r[:,moved_elec,i_r] = np.reshape(np.transpose(wf_gen[ptcls_x,:]),(N_pt,))
                        
                        if inside_A==1:
                            wf_swap[:,wf_inds[ind_1,up_one[i_r]],up_one[i_r]]= wf_gen[ptcls_x,:] 
                            inside_A=0
                        else:
                            wf_swap[:,moved_elec,i_r]= wf_gen[ptcls_x,:]

                        delta = np.zeros(N)
                        delta[coords_r[moved_elec,i_r]] = -1
                        delta[ptcls_x]=1

                        n_occ_r[:,i_r] = n_occ_r[:,i_r] + delta
                        n_pos_r[:,i_r] = n_pos_r[:,i_r] + (moved_elec + 1)*delta
                        coords_r[moved_elec,i_r] = ptcls_x

                x=np.argwhere(n_pos_r[:,i_r]>0)
                assert len(x)== N_pt, 'no of ptcle is %d' % (len(x))
                assert np.sum(n_occ_r[inds_A,i_r])== N_pt_A, 'n_occ_A anc ptcle is %d' % (np.sum(n_occ_r[inds_A,i_r]))
                assert np.sum(n_occ_r[:,i_r])== N_pt, 'n_occ anc ptcle is %d' % (np.sum(n_occ_r[:,i_r]))


# ##############################################################     
            
        if state> (min_state-1):
            
            ratio_r=1.0+0j
            for i_r in range(r):
                ratio_r *= np.linalg.det(wf_swap[:,:,i_r])/np.linalg.det(wf_r[:,:,i_r]) #product of r transposed ratios
            ent_ratio[count] = np.abs(ratio_r)
            count+=1

        
        if (state%500) ==0:
            for i_r in range(r):
                wf_inv_r[:,:,i_r]=np.linalg.inv(wf_r[:,:,i_r])


    acc_ratio=move_accepted/move_attempted
    
    
    #print("Amplitude acceptance rate=", acc_ratio)
    return np.mean(ent_ratio)

In [7]:
def VMC_phase_ratio(numconfig,check_step,N,N_pt,wf_gen,wf_r,n_occ_r,n_pos_r,coords_r,inds_A,N_pt_A):

    move_attempted = 0
    move_accepted = 0
    
    r = wf_r.shape[2]
    
    wf_inv_r = np.zeros(wf_r.shape,dtype=np.complex128)
    wf_inds= np.zeros((N_pt_A,r),dtype=int)
    for i_r in range(r):
        wf_inv_r[:,:,i_r]=np.linalg.inv(wf_r[:,:,i_r])
        pt_num_inside = np.argwhere( n_occ_r[inds_A,i_r]>0 )
        pt_num_inside = np.reshape( pt_num_inside, (N_pt_A,)).tolist()
        wf_inds[:,i_r]=( n_pos_r[ inds_A[pt_num_inside], i_r] )-1    #Used to be sorted
    
    wf_swap = np.zeros(wf_r.shape,dtype=np.complex128)
    wf_swap_inv = np.zeros(wf_r.shape,dtype=np.complex128)
    up_one = (np.arange(r)+1)%r
    down_one = (np.arange(r)-1)%r
    for i_r in range(r):
    # r permutation of subsystem indices
        wf_swap[:,:,i_r] = np.copy(wf_r[:,:,i_r])
        wf_swap[:,wf_inds[:,i_r],i_r] = np.copy(wf_r[:,wf_inds[:,up_one[i_r]],up_one[i_r]]) #Shifting configurations with the next wf in sequence
        wf_swap_inv[:,:,i_r] = np.linalg.inv(wf_swap[:,:,i_r])

    step_abs=np.arange(1,Ns+1) 
    step_vals=np.sort(np.concatenate((-step_abs,step_abs),axis=0)).tolist()

    
    min_state=5
    
    phi_0 = 1
    for i_r in range(r):
        phi_0*=np.conj(np.linalg.det(wf_r[:,:,i_r]))*np.linalg.det(wf_swap[:,:,i_r])
    phi_0= np.angle(phi_0)
        
    phi=phi_0

    ent_ratio=np.zeros(numconfig,dtype=np.complex64) 
    
    for state in range(numconfig+min_state):
        for step in range(0,check_step):               #Samples observables after check_step iterations
        
            for i_r in range(r):
                for moved_elec in range(N_pt):
                    move_attempted=move_attempted+1
                    
                    stepx=random.sample(step_vals,1)[0]
                    ptcls_x= np.mod( coords_r[moved_elec,i_r]+stepx, N) # new configuration

                    if n_occ_r[ptcls_x,i_r]==1:
                        continue

                    inside_A=0    
                    val_orig = np.min( np.abs(inds_A- coords_r[moved_elec,i_r]) )
                    val_dest = np.min( np.abs(inds_A-ptcls_x) )
                    if val_orig==0 and val_dest==0 :
                        inside_A=1
                        val_1 = np.min(np.abs(wf_inds[:,i_r]-moved_elec))
                        ind_1 = np.argmin(np.abs(wf_inds[:,i_r]-moved_elec))
                        assert val_1==0 , 'wrong inds_A for wf'
                        rel = np.dot(wf_swap_inv[wf_inds[ind_1,up_one[i_r]],:,up_one[i_r]],np.transpose(wf_gen[ptcls_x,:])) #down_one[i_r]?
                        #relative = np.conj(np.linalg.det(wf_new_r[:,:,i_r]))*np.linalg.det(wf_new_swap[:,:,i_r])/(np.conj(np.linalg.det(wf_r[:,:,i_r]))*np.linalg.det(wf_swap[:,:,i_r])) 
                    elif val_orig!=0 and val_dest!=0 :
                        rel = np.dot(wf_swap_inv[moved_elec,:,i_r],np.transpose(wf_gen[ptcls_x,:])) #down_one[i_r]?
                        #relative = np.conj(np.linalg.det(wf_new_r[:,:,i_r]))*np.linalg.det(wf_new_swap[:,:,i_r])/(np.conj(np.linalg.det(wf_r[:,:,i_r]))*np.linalg.det(wf_swap[:,:,i_r])) 
                        inside_A=0
                    else:
                        continue
                        
                    u_0 = np.transpose(wf_gen[ptcls_x,:] - wf_gen[coords_r[moved_elec,i_r],:])
                    pt_wf_1=np.transpose(wf_gen[ptcls_x,:])

                    rel =  rel*np.conj(np.dot(wf_inv_r[moved_elec,:,i_r],pt_wf_1))

                    alpha=min(1, np.abs(rel))

                    random_num=random.random()

                    if random_num <= alpha:
                        move_accepted += 1

                        #Sherman-Morisson Formula for updating inverse matrix
                        v=np.zeros((N_pt,1))
                        v[moved_elec]=1
                        num = np.dot(np.dot(np.dot(wf_inv_r[:,:,i_r],u_0),v.T),wf_inv_r[:,:,i_r])
                        denom = (1+np.dot(v.T,np.dot(wf_inv_r[:,:,i_r],u_0)))
                        wf_inv_r[:,:,i_r] = wf_inv_r[:,:,i_r] - num/denom 
                        wf_r[:,moved_elec,i_r] = np.reshape(np.transpose(wf_gen[ptcls_x,:]),(N_pt,))
                        
                        if inside_A==1:
                            u_1=np.reshape(pt_wf_1,(N_pt,1)) - np.reshape(wf_swap[:,wf_inds[ind_1,up_one[i_r]],up_one[i_r]],(N_pt,1))
                            v=np.zeros((N_pt,1))
                            v[wf_inds[ind_1,up_one[i_r]]]=1
                            wf_swap_inv[:,:,up_one[i_r]]=wf_swap_inv[:,:,up_one[i_r]] - np.dot(np.dot(np.dot(wf_swap_inv[:,:,up_one[i_r]],u_1),v.T),wf_swap_inv[:,:,up_one[i_r]]) \
                                 /(1+np.dot(v.T,np.dot(wf_swap_inv[:,:,up_one[i_r]],u_1))) 
                            wf_swap[:,wf_inds[ind_1,up_one[i_r]],up_one[i_r]]= np.reshape(pt_wf_1,(N_pt,))
                            inside_A=0
                        else:
                            u_1=np.reshape(pt_wf_1,(N_pt,1)) - np.reshape(wf_swap[:,moved_elec,i_r],(N_pt,1))
                            wf_swap_inv[:,:,i_r]=wf_swap_inv[:,:,i_r] - np.dot(np.dot(np.dot(wf_swap_inv[:,:,i_r],u_1),v.T),wf_swap_inv[:,:,i_r]) \
                                    /(1+np.dot(v.T,np.dot(wf_swap_inv[:,:,i_r],u_1))) 

                            wf_swap[:,moved_elec,i_r]= np.reshape(pt_wf_1,(N_pt,))
                        
                        delta = np.zeros(N)
                        delta[coords_r[moved_elec,i_r]] = -1
                        delta[ptcls_x]=1

                        n_occ_r[:,i_r] = n_occ_r[:,i_r] + delta
                        n_pos_r[:,i_r] = n_pos_r[:,i_r] + (moved_elec + 1)*delta
                        coords_r[moved_elec,i_r] = ptcls_x
                        
                        phi = phi + np.angle(rel)

                x=np.argwhere(n_pos_r[:,i_r]>0)
                assert len(x)== N_pt, 'no of ptcle is %d' % (len(x))
                assert np.sum(n_occ_r[inds_A,i_r])== N_pt_A, 'n_occ_A anc ptcle is %d' % (np.sum(n_occ_r[inds_A,i_r]))
                assert np.sum(n_occ_r[:,i_r])== N_pt, 'n_occ anc ptcle is %d' % (np.sum(n_occ_r[:,i_r]))
                        



# ##############################################################
    
        if state> (min_state-1):
            #phi = 1
            #for i_r in range(r):
            #    phi*=np.conj(np.linalg.det(wf_r[:,:,i_r]))*np.linalg.det(wf_swap[:,:,i_r])
            #phi= np.angle(phi)
            ent_ratio[state-min_state] = phi

        
        if (state%500) ==0:
            for i_r in range(r):
                wf_inv_r[:,:,i_r]=np.linalg.inv(wf_r[:,:,i_r])
                wf_swap_inv[:,:,i_r] = np.linalg.inv(wf_swap[:,:,i_r])
        #print("Rel is:", rel)


    acc_ratio=move_accepted/move_attempted

        
    #print("Angle acceptance rate=", acc_ratio)
    return np.mean(np.exp(1j*ent_ratio))#

In [8]:
N = 6
N_pt = np.int(N/2)
n_hop = 3                       #maximum length of random walk
numconfig = 10000
check_step =2
Ns=4

t = 1
BC=-1  #np.exp(1j*np.pi/3)
r=2                             #Degree of Renyi entropy
epsilon=  1e-8

Orbitals = wf_gen(N,N_pt,BC,t)

wf_r= np.zeros((N_pt,N_pt,r),dtype=np.complex64)
n_occ_r= np.zeros((N,r),dtype=int)
coords_r= np.zeros((N_pt,r),dtype=int)
n_pos_r = np.zeros((N,r),dtype=int)
    
cond_r= np.zeros(r)
for i_r in range(r):
    n_occ_r[:,i_r], n_pos_r[:,i_r], coords_r[:,i_r]=initialize_wf(N, N_pt)
    wf_r[:,:,i_r]=np.transpose(Orbitals[coords_r[:,i_r],:])
    cond_r[i_r]=np.linalg.cond(wf_r[:,:,i_r])

Lsub_list= np.arange(int(N/2),N,5)
Rr_ex=np.zeros(len(Lsub_list))
Rr_vmc=np.zeros(len(Lsub_list), dtype=np.complex64)
Gmat=np.dot(Orbitals,np.matrix(Orbitals).H)



t_timer=time.time()
random.seed(time.time())
    
for index in range(len(Lsub_list)):
    inds_A = np.arange(Lsub_list[index])    
    Frac = VMC_equal_number(numconfig,check_step,N,N_pt,Orbitals,wf_r,n_occ_r,coords_r,inds_A)
    #print("The fraction for the region",inds_A, "is: ",Frac)

    ################### Checking Amplitude/Phase calculation ##########################

    #N_pt_A = 2
    amp_inds = np.zeros(len(inds_A)+1)
    phase_inds = np.zeros(len(inds_A)+1,dtype=np.complex64)
    for ind in range(len(inds_A)+1):
        N_pt_A = ind
        
        
        if ind < int(len(inds_A)/2-2) or ind > int(len(inds_A)/2+2):
                    continue    #Excludes entries further that +/- 2 from half-filling in region A
                
        check = 0
        miss = 0
        threshold = 100000
        while check < 1 and np.max(cond_r) < 1/epsilon:
            for i_r in range(r):
                n_occ_r[:,i_r], n_pos_r[:,i_r], coords_r[:,i_r]=initialize_wf(N, N_pt)
                wf_r[:,:,i_r]=np.transpose(Orbitals[coords_r[:,i_r],:])
                cond_r[i_r]=np.linalg.cond(wf_r[:,:,i_r])
            if np.max(np.sum(n_occ_r[inds_A,:],axis=0)) == np.min(np.sum(n_occ_r[inds_A,:],axis=0))==N_pt_A:
                check+=1
            elif miss>threshold:
                print('The particle no. %d is vanishingly likely for the subsystem size' %(N_pt_A))
                check +=1 
                miss+=1
                continue
            else:
                miss+=1
        if miss>threshold:
            amp_inds[ind] = 0 
            phase_inds[ind],ent_ratio = 0 ,0
        else:
            amp_inds[ind] = VMC_amplitude_ratio(numconfig,check_step,N,N_pt,Orbitals,wf_r,n_occ_r,n_pos_r,coords_r,inds_A,N_pt_A)
            phase_inds[ind] = VMC_phase_ratio(numconfig,check_step,N,N_pt,Orbitals,wf_r,n_occ_r,n_pos_r,coords_r,inds_A,N_pt_A)
            #print("The amplitude for the region",inds_A, "with ", N_pt_A, " particles is: ", amp_inds[ind])
            #print("The phase for the region",inds_A, "with ", N_pt_A, " particles is: ", phase_inds[ind])
            #print("The fraction for the region",inds_A, "with ", N_pt_A, " particles is: ", Frac[ind])


    ################### Checking Complete calculation ##########################
    Ren = np.sum(Frac*amp_inds*phase_inds) #There should be a break up for each n of paricles in Frac
    
    Rr_ex[index]=exact_renyi_calc(r,Gmat[np.ix_(inds_A,inds_A)])
    Rr_vmc[index] = -np.abs(np.log(Ren))/(1-r)

    print("subsystem size = ", Lsub_list[index])
    print("Approximate %d Renyi entropy: "%(r), Rr_vmc[index-1], " vs exact value", Rr_ex[index-1] )
elapsed = time.time() - t_timer
print("Program finished, elapsed time =", elapsed, "sec")

exact gs energy is -1.732050807568877
subsystem size =  3
Approximate 2 Renyi entropy:  (0.6412102+0j)  vs exact value 0.6567492145180094
Program finished, elapsed time = 37.58159303665161 sec


In [9]:
Frac, amp_inds, phase_inds

(array([0.    , 0.485 , 0.0787, 0.0007]),
 array([0.9999999 , 0.9998325 , 0.5788432 , 0.99999995]),
 array([1.    +0.0000000e+00j, 1.    +0.0000000e+00j,
        0.9008-1.7160123e-07j, 1.    +0.0000000e+00j], dtype=complex64))

In [10]:
def VMC_negativity_equal_number(numconfig,check_step,N,N_pt,WF_gen,wf_r,n_occ_r,coords_r,inds_A1,indsA2,del_occ):


    move_attempted = 0
    move_accepted = 0
    
    r = wf_r.shape[2]
    #inds_A = np.append(inds_A1,inds_A2)
    i_up = (np.arange(r)+1)%r
    i_down = (np.arange(r)-1)%r
    
    wf_inv_r = np.zeros(wf_r.shape,dtype=np.complex128)
    for i_r in range(r):
        wf_inv_r[:,:,i_r]=np.linalg.inv(wf_r[:,:,i_r])
        

    min_state= 5  #Number of full run-throughs before evaluation
    ratio_0=1.0+0.0j
    
    Na = np.zeros((1,r))
    
    n1_occ =  min(len(inds_A1)+1,1+2*del_occ) #The indicies of occupancies in A1 to be explored #len(inds_A1)+1
    n2_occ =  min(len(inds_A2)+1,1+2*del_occ) #The indicies of occupancies in A2 to be explored #len(inds_A2)+1
    #fraction = np.zeros((n1_occ,n2_occ,r))
    #fraction1=np.zeros((n1_occ,r)) #a count for each potential particle number in the region of the i'th copy
    #fraction2=np.zeros((n2_occ,r)) #a count for each potential particle number in the region of the i'th copy
    frac_inds = []
    num1_ind = [n1_occ]; frac1_inds = []
    num2_ind = [n2_occ]; frac2_inds = []
    for i_r in range(r):
        frac_inds = np.append(frac_inds,num1_ind)
        frac_inds = np.append(frac_inds,num2_ind)
    frac_inds = np.array(frac_inds).astype(int)
    Fraction = np.zeros(frac_inds)
    
    
    # possible steps
    step_abs=np.arange(1,Ns+1) #Ns?
    step_vals=np.sort(np.concatenate((-step_abs,step_abs),axis=0)).tolist()

    for state in range(numconfig+min_state):
        for step in range(0,check_step):               #Samples observables after check_step iterations
        
            for i_r in range(r):
                for moved_elec in range(N_pt):
                    move_attempted=move_attempted+1

                    # random walk of Ns steps left or right
                    stepx=random.sample(step_vals,1)[0]
                    ptcls_x= np.mod( coords_r[moved_elec,i_r]+stepx, N) # new configuration

                    if n_occ_r[ptcls_x,i_r]==1:
                        continue

                    u_0 = np.transpose(WF_gen[ptcls_x,:] - WF_gen[coords_r[moved_elec,i_r],:])

                    rel =  np.dot(WF_gen[ptcls_x,:],wf_inv_r[moved_elec,:,i_r].reshape((N_pt,1)))

                    alpha=min(1, np.abs(rel)**2)

                    random_num=random.random()

                    if random_num <= alpha:
                        move_accepted += 1

                        #Sherman-Morisson Formula for updating inverse matrix
                        v=np.zeros((N_pt,1))
                        v[moved_elec]=1
                        num = np.dot(np.dot(np.dot(wf_inv_r[:,:,i_r],u_0),v.T),wf_inv_r[:,:,i_r])
                        denom = (1+np.dot(v.T,np.dot(wf_inv_r[:,:,i_r],u_0)))
                        wf_inv_r[:,:,i_r] = wf_inv_r[:,:,i_r] - num/denom 
                        wf_r[:,moved_elec,i_r] = np.reshape(np.transpose(WF_gen[ptcls_x,:]),(N_pt,))

                        delta = np.zeros(N)
                        delta[coords_r[moved_elec,i_r]] = -1
                        delta[ptcls_x]=1

                        n_occ_r[:,i_r] = n_occ_r[:,i_r] + delta
                        n_pos_r[:,i_r] = n_pos_r[:,i_r] + (moved_elec + 1)*delta
                        coords_r[moved_elec,i_r] = ptcls_x 
                        
                x=np.argwhere(n_occ_r[:,i_r]>0)
                assert len(x)== N_pt, 'no of ptcle is %d' % (len(x))

        if state> (min_state-1):
            
            pt_num_inside_A1= np.sum(n_occ_r[inds_A1,:],axis=0)
            pt_num_inside_A2= np.sum(n_occ_r[inds_A2,:],axis=0)
            pt_num_inside_A= pt_num_inside_A1 + pt_num_inside_A2
            
            Occ_inds = np.array(np.zeros(2*r,dtype = 'int'))
            Occ_A1_inds = np.arange(0,2*r,2)
            Occ_A2_inds = np.arange(1,2*r,2)
                
            check_1 = pt_num_inside_A
            check_2 = pt_num_inside_A1[i_up]+pt_num_inside_A2[i_down]
            comparison = check_1 == check_2
            if np.abs(np.max(pt_num_inside_A1)-int(len(inds_A1)/2)) > del_occ or np.abs(np.max(pt_num_inside_A2)-int(len(inds_A2)/2)) > del_occ:
                continue
            
            elif comparison.all():
                #Occ_inds = np.append(pt_num_inside_A1,pt_num_inside_A2).astype(int)
                Occ_inds[Occ_A1_inds] = pt_num_inside_A1
                Occ_inds[Occ_A2_inds] = pt_num_inside_A2
                Occ_inds = tuple(Occ_inds)
                Fraction[Occ_inds]+=1
                #for i_r in range(r):
                #    fraction1[pt_num_inside_A1[i_r],i_r] += 1
                #    fraction2[pt_num_inside_A2[i_r],i_r] += 1
                #    fraction[pt_num_inside_A1[i_r],pt_num_inside_A2[i_r],i_r] += 1
          
                
        if (state%500) ==0:
            for i_r in range(r):
                wf_inv_r[:,:,i_r]=np.linalg.inv(wf_r[:,:,i_r])

    acc_ratio=move_accepted/move_attempted
    #fraction1 = fraction1/(numconfig)
    #fraction2 = fraction2/(numconfig)
    #fraction = fraction/(numconfig)
    Fraction = Fraction/numconfig
    
    #print("fraction acceptance rate=", acc_ratio)
    return  Fraction

In [11]:
def VMC_negativity_amplitude_ratio(numconfig,check_step,N,N_pt,WF_gen,wf_r,n_occ_r,n_pos_r,coords_r,inds_A1,N_pt_A1,inds_A2,N_pt_A2):

    move_attempted = 0
    move_accepted = 0
    
    r = wf_r.shape[2]
    
    N_pt_A = N_pt_A1+N_pt_A2
    inds_A = np.append(inds_A1,inds_A2)
    i_up = (np.arange(r)+1)%r
    i_down = (np.arange(r)-1)%r
    
    wf_swap = np.zeros(wf_r.shape,dtype=np.complex128)
    wf_inv_r = np.zeros(wf_r.shape,dtype=np.complex128)
    wf_inds_AA =  np.zeros((np.max(N_pt_A),r),dtype=int)
    for i_r in range(r):
        wf_inv_r[:,:,i_r]=np.linalg.inv(wf_r[:,:,i_r])
        
        config_AA = np.argwhere( n_occ_r[inds_A,i_r]>0 )
        config_AA = np.reshape( config_AA, (len(config_AA),)).tolist()
        wf_inds_AA[:N_pt_A[i_r],i_r]= ( n_pos_r[ inds_A[config_AA],i_r ] )-1

        
    inside_A = 0
    step_abs=np.arange(1,Ns+1) #Ns?
    step_vals=np.sort(np.concatenate((-step_abs,step_abs),axis=0)).tolist()
        
    count=0 # counter for entropy
    min_state=5  #Number of full run-throughs before evaluation
    
    neg_ratio=np.zeros(int(numconfig),dtype=np.float64) 
    
    for state in range(numconfig+min_state):
        for step in range(0,check_step):               #Samples observables after check_step iterations
        
            for i_r in range(r):
                for moved_elec in range(N_pt):
                    move_attempted=move_attempted+1
                    
                    stepx=random.sample(step_vals,1)[0]
                    ptcls_x= np.mod( coords_r[moved_elec,i_r]+stepx, N) # new configuration

                    if n_occ_r[ptcls_x,i_r]==1:
                        continue

                    if len(inds_A) == 0:# or N_pt_A1[i_r]==0:
                        val_orig_A = []
                        val_dest_A = []
                    elif len(inds_A) != 0:
                        val_orig_A = np.min( np.abs(inds_A- coords_r[moved_elec,i_r]) )
                        val_dest_A = np.min( np.abs(inds_A-ptcls_x) )
                        
                    if val_orig_A==0 and val_dest_A==0 :   #preserve occupancy in A, but allows transition from A1 to A2
                        inside_A=1
                        val = np.min(np.abs(wf_inds_AA[:N_pt_A[i_r],i_r]-moved_elec))
                        ind = np.argmin(np.abs(wf_inds_AA[:N_pt_A[i_r],i_r]-moved_elec))
                        assert val==0 , 'wrong inds_A1 for wf'
                        
                    elif val_orig_A!=0 and val_dest_A!=0:
                        inside_A=0

                    else:
                        inside_A=0
                        continue

                    u_0 = np.transpose(WF_gen[ptcls_x,:] - WF_gen[coords_r[moved_elec,i_r],:])
                    #pt_wf_1=np.transpose(wf_gen[ptcls_x,:])

                    rel =  np.dot(WF_gen[ptcls_x,:],wf_inv_r[moved_elec,:,i_r].reshape((N_pt,1)))
                    #rel=np.dot(inv_wf[moved_elec,:],pt_wf_1)

                    alpha=min(1, np.abs(rel)**2)

                    random_num=random.random()

                    if random_num <= alpha:
                        move_accepted += 1

                        #Sherman-Morisson Formula for updating inverse matrix
                        v=np.zeros((N_pt,1))
                        v[moved_elec]=1
                        num = np.dot(np.dot(np.dot(wf_inv_r[:,:,i_r],u_0),v.T),wf_inv_r[:,:,i_r])
                        denom = (1+np.dot(v.T,np.dot(wf_inv_r[:,:,i_r],u_0)))
                        wf_inv_r[:,:,i_r] = wf_inv_r[:,:,i_r] - num/denom 
                        wf_r[:,moved_elec,i_r] = np.reshape(np.transpose(WF_gen[ptcls_x,:]),(N_pt,))

                        delta = np.zeros(N)
                        delta[coords_r[moved_elec,i_r]] = -1
                        delta[ptcls_x]=1

                        n_occ_r[:,i_r] = n_occ_r[:,i_r] + delta
                        n_pos_r[:,i_r] = n_pos_r[:,i_r] + (moved_elec + 1)*delta
                        coords_r[moved_elec,i_r] = ptcls_x

                x=np.argwhere(n_pos_r[:,i_r]>0)
                assert len(x)== N_pt, 'no of ptcle is %d' % (len(x))
                assert np.sum(n_occ_r[inds_A,i_r])== N_pt_A[i_r], 'n_occ_A anc ptcle is %d' % (np.sum(n_occ_r[inds_A,i_r]))
                assert np.sum(n_occ_r[:,i_r])== N_pt, 'n_occ anc ptcle is %d' % (np.sum(n_occ_r[:,i_r]))


# ##############################################################     n_occ_r,n_pos_r,coords_r
            
        if state> (min_state-1):
            
            #number_pt_inside_A= np.sum(n_occ_r[inds_A,:],axis=0)

            t_A1 = np.zeros(r,dtype = 'int')
            t_A2 = np.zeros(r,dtype = 'int')
            number_pt_inside_A1= np.sum(n_occ_r[inds_A1,:],axis=0); t_A1 =number_pt_inside_A1
            number_pt_inside_A2= np.sum(n_occ_r[inds_A2,:],axis=0); t_A2 =number_pt_inside_A2
                        
            check_1 = N_pt_A
            check_2 = number_pt_inside_A1[i_up]+number_pt_inside_A2[i_down]
            comparison = check_1 == check_2
            
            if comparison.all():
                ratio_n=1.0+0j
                for i_r in range(r):
                    
                    config_A1_up = np.argwhere( n_occ_r[inds_A1,i_up[i_r]]>0 )
                    config_A1_up = np.reshape( config_A1_up, (len(config_A1_up),)).tolist()
                    wf_inds_up = ( n_pos_r[ inds_A1[config_A1_up],i_up[i_r]  ] )-1

                    config_A2_down = np.argwhere( n_occ_r[inds_A2,i_down[i_r]]>0 )
                    config_A2_down = np.reshape( config_A2_down, (len(config_A2_down),)).tolist()
                    wf_inds_down = ( n_pos_r[ inds_A2[config_A2_down],i_down[i_r]  ] )-1

                    config_A = np.argwhere( n_occ_r[inds_A,i_r]>0 )
                    config_A = np.reshape( config_A, (len(config_A),)).tolist()
                    wf_inds_A= ( n_pos_r[ inds_A[config_A],i_r ] )-1 
                    
                    wf_swap = np.copy(wf_r[:,:,i_r])
                    wf_swap[:,wf_inds_A[0:len(wf_inds_up)]] = np.copy(wf_r[:,wf_inds_up,i_up[i_r]]) #Shifting forward
                    wf_swap[:,wf_inds_A[len(wf_inds_up):len(wf_inds_up)+len(wf_inds_down)]] = np.copy(wf_r[:,wf_inds_down,i_down[i_r]]) #Shifting back
                    
                    #phase_r[i_r] = (-1)**((-1)**(i_r)*((int(t_A2[i_down[i_r]])+int(t_A2[i_r]))%2)/2 +int((t_A2[i_r]+t_A2[i_down[i_r]])*(t_A1[i_r]+t_A1[i_up[i_r]])))
                    
                    ratio_n*=np.linalg.det(wf_swap)/np.linalg.det(wf_r[:,:,i_r])

                    #count+=1
            else:
                ratio_n = 0
            
            #ratio_n=1.0+0j
            #for i_r in range(r):
            #    ratio_n *= np.linalg.det(wf_swap[:,:,i_r])/np.linalg.det(wf_r[:,:,i_r]) #product of r transposed ratios
            neg_ratio[count] = np.abs(ratio_n)
            count+=1

        
        if (state%500) ==0:
            for i_r in range(r):
                wf_inv_r[:,:,i_r]=np.linalg.inv(wf_r[:,:,i_r])

    Neg_ratio = neg_ratio[np.nonzero(neg_ratio)] # only counts entries that satisfy the matching condition
    acc_ratio=move_accepted/move_attempted
    
    
    #print("Amplitude acceptance rate=", acc_ratio)
    return np.mean(Neg_ratio)

In [12]:
def VMC_negativity_phase_ratio(numconfig,check_step,N,N_pt,WF_gen,wf_r,n_occ_r,n_pos_r,coords_r,inds_A1,N_pt_A1,inds_A2,N_pt_A2):

    move_attempted = 0
    move_accepted = 0
    rel_count = 0
    
    r = wf_r.shape[2]
    
    inds_A = np.append(inds_A1,inds_A2)
    i_up = (np.arange(r)+1)%r
    i_down = (np.arange(r)-1)%r
    
    wf_swap = np.zeros(wf_r.shape,dtype=np.complex128)
    wf_swap_inv = np.zeros(wf_r.shape,dtype=np.complex128)
    wf_inv_r = np.zeros(wf_r.shape,dtype=np.complex128)
    wf_inds_A1= np.zeros((np.max(N_pt_A1),r),dtype=int)  #N_pt_A1 should be a list of the r different occupations for A1
    wf_inds_A2= np.zeros((np.max(N_pt_A2),r),dtype=int)
    for i_r in range(r):
        wf_inv_r[:,:,i_r]=np.linalg.inv(wf_r[:,:,i_r])
        
        config_A1 = np.argwhere( n_occ_r[inds_A1,i_r]>0 )
        config_A1 = np.reshape( config_A1, (len(config_A1),)).tolist()
        wf_inds_A1[:N_pt_A1[i_r],i_r]= ( n_pos_r[ inds_A1[config_A1],i_r ] )-1
        
        config_A1_up = np.argwhere( n_occ_r[inds_A1,i_up[i_r]]>0 )
        config_A1_up = np.reshape( config_A1_up, (len(config_A1_up),)).tolist()
        wf_inds_up = ( n_pos_r[ inds_A1[config_A1_up],i_up[i_r]  ] )-1
        
        config_A2 = np.argwhere( n_occ_r[inds_A2,i_r]>0 )
        config_A2 = np.reshape( config_A2, (len(config_A2),)).tolist()
        wf_inds_A2[:N_pt_A2[i_r],i_r]= ( n_pos_r[ inds_A2[config_A2],i_r ] )-1
        
        config_A2_down = np.argwhere( n_occ_r[inds_A2,i_down[i_r]]>0 )
        config_A2_down = np.reshape( config_A2_down, (len(config_A2_down),)).tolist()
        wf_inds_down = ( n_pos_r[ inds_A2[config_A2_down],i_down[i_r]  ] )-1
        
        config_AA = np.argwhere( n_occ_r[inds_A,i_r]>0 )
        config_AA = np.reshape( config_AA, (len(config_AA),)).tolist()
        wf_inds_AA= ( n_pos_r[ inds_A[config_AA],i_r ] )-1
        
        wf_swap[:,:,i_r] = np.copy(wf_r[:,:,i_r])
        wf_swap[:,wf_inds_AA[0:len(wf_inds_up)],i_r] = np.copy(wf_r[:,wf_inds_up,i_up[i_r]]) #Shifting forward
        wf_swap[:,wf_inds_AA[len(wf_inds_up):len(wf_inds_up)+len(wf_inds_down)],i_r] = np.copy(wf_r[:,wf_inds_down,i_down[i_r]])
        wf_swap_inv[:,:,i_r] = np.linalg.inv(wf_swap[:,:,i_r])


    step_abs=np.arange(1,Ns+1) #Ns?
    step_vals=np.sort(np.concatenate((-step_abs,step_abs),axis=0)).tolist()

    
    min_state=5 #Number of full run-throughs before evaluation
    
    phase_r= np.zeros(r,dtype="complex")
    phi_0 = 1
    for i_r in range(r):
        phase_r[i_r] = (-1)**((-1)**(i_r)*((int(N_pt_A2[i_down[i_r]])+int(N_pt_A2[i_r]))%2)/2 +int((N_pt_A2[i_r]+N_pt_A2[i_down[i_r]])*(N_pt_A1[i_r]+N_pt_A1[i_up[i_r]])))
        phi_0*=np.conj(np.linalg.det(wf_r[:,:,i_r]))*np.linalg.det(wf_swap[:,:,i_r])
    phi_0= np.angle(phi_0)
        
    phi=phi_0

    neg_ratio=np.zeros(numconfig,dtype=np.complex64) 
    neg_ratio1=np.zeros(numconfig,dtype=np.complex64) 
    
    for state in range(numconfig+min_state):
        for step in range(0,check_step):               #Samples observables after check_step iterations
        
            for i_r in range(r):
                for moved_elec in range(N_pt):
                    move_attempted=move_attempted+1
                    
                    stepx=random.sample(step_vals,1)[0]
                    ptcls_x= np.mod( coords_r[moved_elec,i_r]+stepx, N) # new configuration

                    if n_occ_r[ptcls_x,i_r]==1:
                        continue
                    
                    if len(inds_A1) == 0:# or N_pt_A1[i_r]==0:
                        val_orig_A1 = []
                        val_dest_A1 = []
                    elif len(inds_A1) != 0:
                        val_orig_A1 = np.min( np.abs(inds_A1- coords_r[moved_elec,i_r]) )
                        val_dest_A1 = np.min( np.abs(inds_A1-ptcls_x) )
                    if len(inds_A2) == 0:# or N_pt_A2[i_r]==0:
                        val_orig_A2 = []
                        val_dest_A2 = []
                    elif len(inds_A2) != 0:
                        val_orig_A2 = np.min( np.abs(inds_A2- coords_r[moved_elec,i_r]) )
                        val_dest_A2 = np.min( np.abs(inds_A2-ptcls_x) )
                    
                    inside_A1 = 0
                    inside_A2 = 0
                    if val_orig_A1==0 and val_dest_A1==0 :
                        inside_A1=1; inside_A2=0
                        val_1 = np.min(np.abs(wf_inds_A1[:N_pt_A1[i_r],i_r]-moved_elec))
                        ind_1 = np.argmin(np.abs(wf_inds_A1[:N_pt_A1[i_r],i_r]-moved_elec))
                        assert val_1==0 , 'wrong inds_A1 for wf'
                        rel = np.dot(wf_swap_inv[wf_inds_A1[ind_1,i_up[i_r]],:,i_up[i_r]],np.transpose(WF_gen[ptcls_x,:])) 
                    elif val_orig_A2==0 and val_dest_A2==0 :
                        inside_A2=1; inside_A1=0
                        val_2 = np.min(np.abs(wf_inds_A2[:N_pt_A2[i_r],i_r]-moved_elec))
                        ind_2 = np.argmin(np.abs(wf_inds_A2[:N_pt_A2[i_r],i_r]-moved_elec))
                        assert val_2==0 , 'wrong inds_A2 for wf'
                        rel = np.dot(wf_swap_inv[wf_inds_A2[ind_2,i_down[i_r]],:,i_down[i_r]],np.transpose(WF_gen[ptcls_x,:])) 
                    elif val_orig_A1!=0 and val_dest_A1!=0 and val_orig_A2!=0 and val_dest_A2!=0 :
                        rel = np.dot(wf_swap_inv[moved_elec,:,i_r],np.transpose(WF_gen[ptcls_x,:])) 
                        inside_A1=0
                        inside_A2=0
                    else:
                        inside_A1=0
                        inside_A2=0
                        continue
                        
                    u_0 = np.transpose(WF_gen[ptcls_x,:] - WF_gen[coords_r[moved_elec,i_r],:])
                    pt_wf_1=np.transpose(WF_gen[ptcls_x,:])

                    rel =  rel*np.conj(np.dot(wf_inv_r[moved_elec,:,i_r],pt_wf_1))

                    alpha=min(1, np.abs(rel))

                    random_num=random.random()

                    if random_num <= alpha:
                        move_accepted += 1
                        rel_count += rel

                        #Sherman-Morisson Formula for updating inverse matrix
                        v=np.zeros((N_pt,1))
                        v[moved_elec]=1
                        num = np.dot(np.dot(np.dot(wf_inv_r[:,:,i_r],u_0),v.T),wf_inv_r[:,:,i_r])
                        denom = (1+np.dot(v.T,np.dot(wf_inv_r[:,:,i_r],u_0)))
                        wf_inv_r[:,:,i_r] = wf_inv_r[:,:,i_r] - num/denom 
                        wf_r[:,moved_elec,i_r] = np.reshape(np.transpose(WF_gen[ptcls_x,:]),(N_pt,))
                        
                        if inside_A1==1:
                            u_1=np.reshape(pt_wf_1,(N_pt,1)) - np.reshape(wf_swap[:,wf_inds_A1[ind_1,i_up[i_r]],i_up[i_r]],(N_pt,1))
                            v=np.zeros((N_pt,1))
                            v[wf_inds_A1[ind_1,i_up[i_r]]]=1
                            num_1 = np.dot(np.dot(np.dot(wf_swap_inv[:,:,i_up[i_r]],u_1),v.T),wf_swap_inv[:,:,i_up[i_r]])
                            denom_1 = 1+np.dot(v.T,np.dot(wf_swap_inv[:,:,i_up[i_r]],u_1))
                            wf_swap_inv[:,:,i_up[i_r]]=wf_swap_inv[:,:,i_up[i_r]] - num_1/denom_1 
                            wf_swap[:,wf_inds_A1[ind_1,i_up[i_r]],i_up[i_r]]= np.reshape(pt_wf_1,(N_pt,))
                            inside_A1=0
                            assert denom_1!=0, "and v, coords, ptcls_x and det_swap %d" (v,coords_r[moved_elec,i_r], ptcls_x, np.linalg.det(wf_swap[:,:,i_r]))
                        elif inside_A2==1:
                            u_1=np.reshape(pt_wf_1,(N_pt,1)) - np.reshape(wf_swap[:,wf_inds_A2[ind_2,i_down[i_r]],i_down[i_r]],(N_pt,1))
                            v=np.zeros((N_pt,1))
                            v[wf_inds_A2[ind_2,i_down[i_r]]]=1
                            num_2 = np.dot(np.dot(np.dot(wf_swap_inv[:,:,i_down[i_r]],u_1),v.T),wf_swap_inv[:,:,i_down[i_r]])
                            denom_2 = 1+np.dot(v.T,np.dot(wf_swap_inv[:,:,i_down[i_r]],u_1))
                            wf_swap_inv[:,:,i_down[i_r]]= wf_swap_inv[:,:,i_down[i_r]] - num_2/denom_2 
                            wf_swap[:,wf_inds_A2[ind_2,i_down[i_r]],i_down[i_r]]= np.reshape(pt_wf_1,(N_pt,))
                            inside_A2=0
                        else:
                            u_1=np.reshape(pt_wf_1,(N_pt,1)) - np.reshape(wf_swap[:,moved_elec,i_r],(N_pt,1))
                            wf_swap_inv[:,:,i_r]=wf_swap_inv[:,:,i_r] - np.dot(np.dot(np.dot(wf_swap_inv[:,:,i_r],u_1),v.T),wf_swap_inv[:,:,i_r]) \
                                    /(1+np.dot(v.T,np.dot(wf_swap_inv[:,:,i_r],u_1))) 

                            wf_swap[:,moved_elec,i_r]= np.reshape(pt_wf_1,(N_pt,))
                        
                        delta = np.zeros(N)
                        delta[coords_r[moved_elec,i_r]] = -1
                        delta[ptcls_x]=1

                        n_occ_r[:,i_r] = n_occ_r[:,i_r] + delta
                        n_pos_r[:,i_r] = n_pos_r[:,i_r] + (moved_elec + 1)*delta
                        coords_r[moved_elec,i_r] = ptcls_x
                        
                        phi = phi + np.angle(rel) 

                x=np.argwhere(n_pos_r[:,i_r]>0)
                assert len(x)== N_pt, 'no of ptcle is %d' % (len(x))
                assert np.sum(n_occ_r[inds_A1,i_r])== N_pt_A1[i_r], 'n_occ_A1 anc ptcle is %d' % (np.sum(n_occ_r[inds_A1,i_r]))
                assert np.sum(n_occ_r[inds_A2,i_r])== N_pt_A2[i_r], 'n_occ_A2 anc ptcle is %d' % (np.sum(n_occ_r[inds_A2,i_r]))
                assert np.sum(n_occ_r[:,i_r])== N_pt, 'n_occ anc ptcle is %d' % (np.sum(n_occ_r[:,i_r]))
                        



# ##############################################################
        #assert n_occ_r[:,i_r].sum()==N_pt, "total particle number changes!!"
    
        if state> (min_state-1):
            psi = 1
            for i_r in range(r):
                psi*=phase_r[i_r]*np.conj(np.linalg.det(wf_r[:,:,i_r]))*np.linalg.det(wf_swap[:,:,i_r])
            psi= np.angle(psi)
            neg_ratio1[state-min_state] = psi  #Includes fermion phase
            neg_ratio[state-min_state] = phi   #w/o fermion phase

        
        if (state%500) ==0:
            for i_r in range(r):
                wf_inv_r[:,:,i_r]=np.linalg.inv(wf_r[:,:,i_r])
                wf_swap_inv[:,:,i_r] = np.linalg.inv(wf_swap[:,:,i_r])


    acc_ratio=move_accepted/move_attempted
    if move_accepted>0:
        rel_avg = rel_count/move_accepted
    else:
        rel_avg = 1


        
    #print("Angle acceptance rate=", acc_ratio)
    return np.mean(np.exp(1j*neg_ratio1)),rel_avg #, use neg_ratio if bosonic

In [14]:
def Ren_sectors(N1,N2,r,del_occ):
    l1 = min(N1,1+2*del_occ) #Minimum out of the max occupancy of an interval and 2*del_occ (interval around half_occupancy)
    l2 = min(N2,1+2*del_occ)
    L1 = np.arange(int(N1/2) - int(l1/2) ,int(N1/2) - int(l1/2)+l1,1, dtype = 'int')
    L2 = np.arange(int(N2/2) - int(l2/2) ,int(N2/2) - int(l2/2)+l2,1, dtype = 'int')
    empt = np.zeros(0,dtype='int')
    indicies =  [np.append(empt,x) for x in itertools.product(L1, L2,repeat = r)]
    
    i_up = (np.arange(r)+1)%r
    i_down = (np.arange(r)-1)%r
    
    list_r = []
    list_n = []
    
    n_A = np.zeros(r,dtype = 'int')
    n_A1 = np.zeros(r,dtype = 'int')
    n_A2 = np.zeros(r,dtype = 'int')
    factor = np.zeros(2*r,dtype = 'int')
    for ind in indicies:
        factor = ind
        for i_r in range(r):
            n_A1[i_r] = factor[i_r*2]
            n_A2[i_r] = factor[i_r*2+1]
            n_A[i_r] = factor[i_r*2] + factor[i_r*2+1]

        check_1 = n_A
        check_2 = n_A1[i_up]+n_A2[i_down]
        comparison = check_1 == check_2
            
        if comparison.all():
            I_r = factor
            list_r.append(np.array(factor))
            list_n.append(np.array(n_A))
            
    return list_r,list_n

del_occ= 2 # tolerace around half filling (use 1 or 2)
r = 2      # renyi index
len_A1=2   # 1 + length of Region A1
len_A2=3   # 1 + length of Region A1
Occ_list,Ren_sect = Ren_sectors(len_A1,len_A2,r,del_occ)

print(Occ_list, Ren_sect)

[array([0, 0, 0, 0]), array([0, 1, 0, 1]), array([0, 1, 1, 0]), array([0, 2, 0, 2]), array([0, 2, 1, 1]), array([1, 0, 0, 1]), array([1, 0, 1, 0]), array([1, 1, 0, 2]), array([1, 1, 1, 1]), array([1, 2, 1, 2])] [array([0, 0]), array([1, 1]), array([1, 1]), array([2, 2]), array([2, 2]), array([1, 1]), array([1, 1]), array([2, 2]), array([2, 2]), array([3, 3])]


In [22]:
t_timer=time.time()
random.seed(time.time())

N = 6
N_pt = np.int(N/2)
n_hop = 3                       #maximum length of random walk
numconfig = 10000
check_step =2
Ns=3

t = 1
BC=-1  #np.exp(1j*np.pi/3)
r=3                             #Degree of Renyi entropy

Orbitals = wf_gen(N,N_pt,BC,t)

wf_r= np.zeros((N_pt,N_pt,r),dtype=np.complex64)
n_occ_r= np.zeros((N,r),dtype=int)
coords_r= np.zeros((N_pt,r),dtype=int)
n_pos_r = np.zeros((N,r),dtype=int)
    
cond_r= np.zeros(r)
for i_r in range(r):
    n_occ_r[:,i_r], n_pos_r[:,i_r], coords_r[:,i_r]=initialize_wf(N, N_pt)
    wf_r[:,:,i_r]=np.transpose(Orbitals[coords_r[:,i_r],:])
    cond_r[i_r]=np.linalg.cond(wf_r[:,:,i_r])

Lsub_list= np.arange(N_pt,N_pt+1,2)                                     #Lengths of region A
#LsubA_list= np.arange(0,6,1)                                        #Lengths of Region A1
Nr_ex=np.zeros((len(Lsub_list),len(Lsub_list)), dtype=np.complex64)           #2nd entry should be len(LsubA_list)
Nr_vmc=np.zeros((len(Lsub_list),len(Lsub_list)), dtype=np.complex64) #int(max(Lsub_list)/2)+1
Gmat=np.dot(Orbitals,np.matrix(Orbitals).H)
threshold = 1e3
epsilon= 1e-8

i_up = (np.arange(r)+1)%r
i_down = (np.arange(r)-1)%r

for index in range(len(Lsub_list)):
    Lsub=Lsub_list[index]
    print('subsystem size = ', Lsub)
    inds_A= np.arange(0,Lsub)
    LsubA_list= [int(Lsub/2)]#np.arange(int(Lsub/2))
    
    for index_A in range(len(LsubA_list)):
        
        LsubA = LsubA_list[index_A]
        inds_A1= np.arange(0,LsubA)
        inds_A2= np.arange(LsubA,Lsub) 
        
        print('partial-transpose region size = ', len(inds_A2))
   
        del_occ = 2 # how far one samples from half-filling
        Frac = VMC_negativity_equal_number(numconfig,check_step,N,N_pt,Orbitals,wf_r,n_occ_r,coords_r,inds_A1,inds_A2,del_occ)
        #print("The fraction for the region",inds_A1, "and region",inds_A2,"is: ",Frac)       
        
        amp_inds = np.zeros(np.shape(Frac)) #frac_inds
        phase_inds = np.zeros(np.shape(Frac),dtype=np.complex64)
        phase_inds1 = np.ones(np.shape(Frac),dtype=np.complex64)

        #amp_inds = np.zeros((len(inds_A1)+1,len(inds_A2)+1,r))#np.zeros(np.max(inds_A)+2)
        #phase_inds = np.ones((len(inds_A1)+1,len(inds_A2)+1,r),dtype=np.complex64) #np.zeros(np.max(inds_A)+2,dtype=np.complex64)
        
        n1_occ = min(len(inds_A1)+1,1+2*del_occ) #The indicies of occupancies in A1 to be explored
        n2_occ = min(len(inds_A2)+1,1+2*del_occ) #The indicies of occupancies in A2 to be explored 
        

        
        #Occ_list = Ren_index(len(inds_A1)+1,len(inds_A2)+1,r, del_occ) #Enumerates the different permutations of occupations for r copies
        Occ_list, Ren_sect = Ren_sectors(len(inds_A1)+1,len(inds_A2)+1,r,del_occ)
        k = len(Occ_list)                    #Same length as Ren_sect
        n_A1 = np.zeros(k,dtype='int')       #occupancies for A_1 in each configuration
        n_A2 = np.zeros(k,dtype='int')       #occupancies for A_1 in each configuration
        n1_ind = np.arange(0,2*r,2)          #indicies of elements in A_1
        n2_ind = np.arange(1,2*r,2)          #indicies of elements in A_1
        
        Amp_inds = np.zeros(k)
        Phase_inds = np.zeros(k,dtype=np.complex64)
        Rel_inds = np.zeros(k,dtype=np.complex64)
        fraction = np.zeros(k)
        
        uniques = []                              #Elements with distinct n_A (true sector decomposition)
        for arr in Ren_sect:
            if not any(np.array_equal(arr, unique_arr) for unique_arr in uniques):
                uniques.append(arr)
        len_u = len(uniques)

        fr_u = np.zeros(len_u)
        amp_u = np.zeros(len_u)
        phase_u = np.zeros(len_u,dtype=np.complex64)

        for i in range(len_u):
            check_i = uniques[i]
            occ_i = tuple(check_i)
            rel_count = 0
            count=0
            for j in range(k):
                check_j = Ren_sect[j]
                occ_j = tuple(check_j)
                comparison = check_i == check_j     #averaging over configurations with the same n_A
                if comparison.all():         
                    occ = Occ_list[j]
                    n_ind = tuple(occ)
                    n_A1 = occ[n1_ind] #occupancy in region A1 for r copies
                    n_A2 = occ[n2_ind] #occupancy in region A2 for r copies
                    n_A = n_A1+n_A2
                    if np.max(n_A)>N_pt:
                        continue
                    if N_pt - min(n_A) > N - len(inds_A1) - len(inds_A2):
                        continue
                        
                    assert n_A[1] == n_A1[i_up[1]]+n_A2[i_down[1]], 'matching failed'    
                        
                    count+=1       #counts number of configurations in sector with the same n_A
                    fraction[j] = Frac[n_ind]
                    fr_u[i] += Frac[n_ind]
                    
                    check = 1/epsilon+1
                    miss = 0
                    while check > 1/epsilon and miss<threshold:
                        for i_r in range(r):
                            n_occ_r[:,i_r], n_pos_r[:,i_r], coords_r[:,i_r]=initialize_wf_region(N, N_pt, inds_A1, n_A1[i_r],inds_A2, n_A2[i_r])
                            wf_r[:,:,i_r]=np.transpose(Orbitals[coords_r[:,i_r],:])
                            cond_r[i_r]=np.linalg.cond(wf_r[:,:,i_r])
                        check = np.max(cond_r)
                        miss +=1

                    if miss>threshold:
                        print("oops....")
                        Phase_inds[j] = 0
                        Amp_inds[j] = 0

                    else:
                        Phase_inds[j], Rel_inds[j]= VMC_negativity_phase_ratio(numconfig,check_step,N,N_pt,Orbitals,wf_r,n_occ_r,n_pos_r,coords_r,inds_A1,n_A1,inds_A2,n_A2)
                        Amp_inds[j]= VMC_negativity_amplitude_ratio(numconfig,check_step,N,N_pt,Orbitals,wf_r,n_occ_r,n_pos_r,coords_r,inds_A1,n_A1,inds_A2,n_A2)

                    amp_u[i] += Amp_inds[j]
                    phase_u[i] += Rel_inds[j]*Phase_inds[j] #Phase value for (n_A1,n_A2) weighted
                    rel_count += Rel_inds[j]     #Total weight within the sector
                else:
                    continue
            amp_u[i] = amp_u[i]/count;
            phase_u[i] = phase_u[i]/rel_count  #Normalized Phase value


        
    ################### Checking Complete calculation ##########################
        #Neg = np.sum(Frac*amp_inds*phase_inds) #There should be a break up for each n of paricles in Frac
        Neg = np.sum(fr_u*amp_u*phase_u)
        Neg1 = np.sum(fraction*Amp_inds*Phase_inds)
        #print("Frac", np.sum(Frac), "amp", np.sum(amp_inds), "phase", np.sum(phase_inds), "alt phase", np.sum(phase_inds1))
        #Nr_ex[index,index_A]=exact_negativity_calc(r,N,N_pt,Orbitals,Lsub,len(inds_A2),epsilon=1e-9)
        Nr_ex[index,index_A] = exact_Renyi_neg_calc(r,N,LsubA,len(inds_A2),Orbitals,epsilon=1e-9)
        Nr_vmc[index,index_A] = -np.abs(np.log(Neg)) #/(1-r)

        print("Approximate %d Negativity: "%(r),Nr_vmc[index,index_A] , " vs exact value",  Nr_ex[index,index_A])
        print("Approximate alternate %d Negativity: "%(r),-np.abs(np.log(Neg1)) )
elapsed = time.time() - t_timer
print("Program finished, elapsed time =", elapsed, "sec")

exact gs energy is -1.732050807568877
subsystem size =  3
partial-transpose region size =  2
Approximate 3 Negativity:  (-1.5616465+0j)  vs exact value (-1.5451722+0j)
Approximate alternate 3 Negativity:  -1.5632386331075372
Program finished, elapsed time = 227.1024887561798 sec
